<a href="https://colab.research.google.com/github/pleunipennings/CSC508_ML_Biomedicine_Class/blob/main/Module6/Module_6b_Gradient_Boosted_Tree_Antibiotic_resistance_dataset_CSC508.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Welcome to the module 6b coding part: Grading Boosted Tree!**

*This notebook was created at San Francisco State University (SFSU) for the Promoting INclusivity and Computing (PINC) and gSTAR programs by Dr. Pleuni Pennings (SFSU biology professor), Lucy Moctezuma Tan (California State University, East Bay CSUEB master student) and Lorena Benitez-Rivera (SFSU master students). All members of the COde to understand Drug resistance Evolution (CODE) lab in 2023.*

#OBJECTIVE OF THIS NOTEBOOK:

In this notebook we will learn how to create a **Gradient Boosted Tree** model using different functions that will allow us to get predicted labels for each type of drug instead of doing it one drug at a time.

The objective of this notebook is to learn how to apply what we have been learning in a more complex project, so you can think of different ways in which machine learning could be used. In this case we will try to recreate specifically some of the results from the **Gradient Boosted Tree** model that was used in [Moragadivand's 2018 paper](https://doi.org/10.1371/journal.pcbi.1006258).

In this notebook you will learn:

- How to create different functions that will let us run our **Gradient Boosted Tree** model per drug and per feature combinations.
- How to link all functions together to create a cohesive workflow and get our results.    

## **Step 1) Importing packages needed**
Just like before we will be loading up the previous packages and functions we have seen so far.

In [ ]:
# Set the scikit-learn version to ensure compatibility
!pip install scikit-learn==1.5.2

In [ ]:
# Data manipulation imports for ML
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Import packages for Gradient Boosted Tree model
from xgboost import XGBClassifier
from sklearn import preprocessing

# Imports for model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Imports for data visualization
import matplotlib.pyplot as plt

# Imports for file management
import os
from google.colab import drive
drive.mount('/content/drive')

## **Step 2) Loading CSV file and creating dataframes for each antibiotic**

#### **a) Loading CSV created from previous notebook (6a)**

Let's first load our merged dataframe, created by notebook 6a, containing each feature and labels for all our drugs. The datataframe contains:

1.   **Year features as a matrix**
2.   **Population structure features**
3.   **Presence and absence of specific genes**

We will learn in the following steps how to select different combinations of these 3 features.

In [ ]:
# Loads csv file as a dataframe
filepath= '/content/drive/MyDrive/EColi_ML_CSV_files/'
# reads csv file as a dataframe
All_Drugs_df = pd.read_csv(filepath+"EColi_Merged_dfs.csv", na_values="NaN")
All_Drugs_df.head()

#### **b) Creating dataframes for each drug**

Below we will be creating the function **makeDF**, this will allow us to create one dataframe for each antibiotic drug, by joining the labels of one group and the features corresponding to it.

In [ ]:
# creating a list of antibiotic names
drug_list = All_Drugs_df.iloc[:,1:13].columns
drug_list

In [ ]:
# creating a function that makes dataframes for each antibiotic and dropping NaN values
def makeDF(drug):
  df_list = [All_Drugs_df[["Isolate",drug]],All_Drugs_df.iloc[:,13:]]
  Drug_df = pd.concat(df_list, axis=1)
  Drug_df = Drug_df.dropna()
  return Drug_df

Below we will demonstrate how **makeDF** works by creating a single dataframe using the the drug AMP that contains the labels for AMP and all the features we are interested in.

In [ ]:
# implementing function using as example the drug AMP
AMP_df = makeDF("AMP")

# looking at the shape of AMP dataframe
print("AMP dataframe shape: ", AMP_df.shape)

# looking at the first 5 rows of this dataframe
AMP_df.head()

## **Step 3) Separating each drug dataframe into sections**

The dataframe will be separated into 4 sections:

1.   **Training features**
2.   **Training labels**
3.   **Testing features**
4.   **Testing labels**








#### **a) Creating testing and training datasets for each antibiotic drug**
Below we will be creating the function **Split_train_test**, which will allow us to split the dataset of each of the dataframes created by our previous function, and store these splits into a single python dictionary. Then we will test this function by splitting only the dataframe we created before using the drug AMP.

In [ ]:
# Separating each dataframe into Labels and Features
def Split_train_test(Drug_df,drug):
  Train_test_dic = {}
  labels = Drug_df[drug]
  features = Drug_df.drop(columns=[drug])
  features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

  Train_test_dic['labels_train'] = labels_train
  Train_test_dic['features_train'] = features_train
  Train_test_dic['labels_test'] = labels_test
  Train_test_dic['features_test'] = features_test

  return Train_test_dic

## Task 1:
Below we implement the function we just created and accessed specifically the training features of the AMP dataset. Try accessing the other three key-value pairs of this dictionary, such as the features of the testing dataset.
What do you think are some of the benefits of using a dictionary to store the different parts of the dataset?

In [ ]:
# Implementing the function Split_train_test() for AMP example
AMP_Train_test_dic = Split_train_test(AMP_df,"AMP")
AMP_Train_test_dic["features_train"]

In [ ]:
# Access the features testing of the dictionary

Below you can also access all the Python dictionary elements we have created for this specific AMP dataset. Below I decided to print the shape for each of the data chunks we have split.

In [ ]:
# checking the shape of each dataframe or series stored in the dictionary created for drug AMP
print("AMP")
for k, df in AMP_Train_test_dic.items():
  print(k, df.shape)

## **Step 4) Creating different combination of features before training**
Below we will be creating the function **combo_feat**, which will allow us to choose what are the specific combinations we want to use to train our model. As you can see we have decided to test all the ones present in the **combo_list** list:

*   S (Population Structure)
*   GS (Gene Presence/Absence and Population Structure)
*   GY (Gene Presence/Absence, Year)
*   SY (Population Structure, Year)
*   GYS (All- Gene Presence/Absence, Population Structure, Year)


In [ ]:
# making a list of combinations of data sources we would like to test in our ML models
combo_list = ['G','S', 'SY', 'GS', 'GYS']

# making a function that creates different feature combinations of the predictor features
def combo_feat(features_df, drug, combo):

  # creating Year column filters for features_df
  year_filter = [col for col in features_df if col.startswith("Year")]
  year_feat = features_df[year_filter]

  # creating Population structure column filters for features_df
  pop_str_filter = [col for col in features_df if col.startswith("cutoff")]
  pop_struc_feat = features_df[pop_str_filter]

  # creating Gene precence column filters for features_df
  gene_presc_filter = [col for col in features_df.columns if col not in pop_str_filter and col not in year_filter and col != "Isolate"]
  gene_presc_feat = features_df[gene_presc_filter]

  if combo == 'G':
    df_list = [features_df['Isolate'], gene_presc_feat]
    G_feat_df = pd.concat(df_list, axis=1)
    G_feat_df = G_feat_df.drop(columns=['Isolate'])
    return G_feat_df

  if combo == 'S':
    df_list = [features_df['Isolate'], pop_struc_feat]
    S_feat_df = pd.concat(df_list, axis=1)
    S_feat_df = S_feat_df.drop(columns=['Isolate'])
    return S_feat_df

  if combo == 'GY':
    df_list = [features_df['Isolate'], gene_presc_feat, year_feat]
    GY_feat_df = pd.concat(df_list, axis=1)
    GY_feat_df = GY_feat_df.drop(columns=['Isolate'])
    return GY_feat_df

  if combo== "GS":
    df_list = [features_df['Isolate'], gene_presc_feat, pop_struc_feat]
    GS_feat_df = pd.concat(df_list, axis=1)
    GS_feat_df = GS_feat_df.drop(columns=['Isolate'])
    return GS_feat_df

  if combo == 'SY':
    df_list = [features_df['Isolate'], pop_struc_feat, year_feat]
    SY_feat_df = pd.concat(df_list, axis=1)
    SY_feat_df = SY_feat_df.drop(columns=['Isolate'])
    return SY_feat_df

  if combo == 'GYS':
    df_list = [features_df['Isolate'], gene_presc_feat, pop_struc_feat, year_feat]
    GYS_feat_df = pd.concat(df_list, axis=1)
    GYS_feat_df = GYS_feat_df.drop(columns=['Isolate'])
    return GYS_feat_df


Following the example of working with a single drug AMP, we can see below that we can actually test and access a specific part of the dictionary we created in the previous step and also choose specifically what feature combination we are interested in training. In this case is **GS (Gene Presence and Absence and Population Structure)**

In [ ]:
# Implementing combo_feat() function created for training data
AMP_GS_train_df = combo_feat(AMP_Train_test_dic['features_train'],"AMP","GS")

# looking only at the feature column names for the combination for "GS" for drug "AMP" for training data
AMP_GS_train_df.columns

## **Step 5) Creating Gradient Boosted Trees model and training it per feature combination**

Below we will create  another function called **run_GB** that will let us create and train our Gradient Boosted Tree, by assigning a training data (features, labels and what feature combination) we would like to train.

In [ ]:
# creating Gradient Boosted Trees model function
def run_GB(feat_train_df, lab_train, drug, combo):
  labels = lab_train
  le = preprocessing.LabelEncoder()
  le.fit(labels)
  labels_t = le.transform(labels)
  print(drug +" Training combo: "+ combo)
  GB =  XGBClassifier(random_state = 42)
  GB = GB.fit(feat_train_df, labels_t)
  return GB

Below we will test our function by:
- Using the **training features** we specified in the previously **(step 4)**.
- Getting our **training labels** from the python dictionary we created before **(step 3)**
- Specifying the feature combination we want, in this example, **GS**.

In [ ]:
# implementing run_GB() for specific drug feature combination dataframe
GB_AMP_GS_model = run_GB(AMP_GS_train_df, AMP_Train_test_dic['labels_train'],"AMP","GS")
GB_AMP_GS_model

## **Step 6) Making predictions from Gradient Boosted Trees model**
We will create here another function called **predict** to make our predictions using:
- The model we trained on **(step 5)**
- Choosing the test features chunks we made from **(step 3)**

In [ ]:
# creating a function using the model created and trained and the feature combinations from testing data
def predict(GB_combo_Model, features_test):
  labels_pred = GB_combo_Model.predict(features_test)
  return labels_pred

Below we will be resusing our function **combo_feat** to specify that we want to use only specific features from out test dataset, because it has to be based on what our model was trained with. In this case we only want **GS combination**.

In [ ]:
# Implementing combo_feat() function created for testing data
AMP_GS_test_df = combo_feat(AMP_Train_test_dic['features_test'],"AMP","GS")

# looking only at the feature column names for the combination for "GS" for drug "AMP" for testing data
AMP_GS_test_df.columns

Finally we will use our function with the correct model and correct testing dataset, where the only combination we want to make predictions with is GS. We will also print out how many did our model predict Resistance(R) and how many were predicted as Susceptible(S). In this case we got 210 Resistant E.Coli Predicted and 68 E.Coli Susceptible Predicted to the AMP drug, using only the feature combination GS.



In [ ]:
# Implementation of the predict() function using the feature combination "GS"
AMP_GS_labels_pred = predict(GB_AMP_GS_model,AMP_GS_test_df)

# transforming back our labels for interpretation in the next output
labels_pred = np.where(AMP_GS_labels_pred<1,"R","S")

# observe how many predictions were made for each category "R" and "S"
print("Labels predicted: ", np.unique(labels_pred, return_counts=True))

## **Step 7) Evaluating our model using a confusion matrix and metrics**

Below we create our last function **evaluate**, where we are able to extract our accuracy and recall for Resistant and Susceptible E.Coli, plus a Confusion Matrix. Notice that within the function, we had to convert the labels test into numbers as well in order to be able to be compared with our predicted labels.

In [ ]:
# Creating a function that evaluates our model using our actual and predicted data
def evaluate(GB_combo_model, labels_test, labels_pred, cf= True):
  labels = labels_test
  le = preprocessing.LabelEncoder()
  le.fit(labels)
  labels_t = le.transform(labels)
  report = classification_report(labels_t, labels_pred, output_dict = True)
  accuracy = report['accuracy']
  R_recall = report["0"]['recall']# Resistant
  S_recall = report["1"]['recall']# Susceptible
  if cf == True:
    labels_pred = np.where(labels_pred<1,"R","S")
    cm = confusion_matrix(labels_test, labels_pred, labels=np.where(GB_combo_model.classes_<1,"R","S"))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.where(GB_combo_model.classes_<1,"R","S"))
    disp.plot()
    plt.show()
  return [accuracy,R_recall,S_recall]

Below we will be testing our function using:
- The model we trained in **(step 5)**
- The test labels we separated and put into a data dictionary in **(Step 3)**
- The predicted labels that we got from **(Step 6)**

In [ ]:
# implementing the evaluate() function
Model_Report = evaluate(GB_AMP_GS_model, AMP_Train_test_dic['labels_test'],AMP_GS_labels_pred)
print("Results from Model for drug: AMP")
print("Using feature combination: GS")
print("Accuracy: ", Model_Report[0])
print("R_recall: ", Model_Report[1])
print("S_recall: ", Model_Report[2])

## Task 2:
As a review, list all the functions we have created so far in this notebook. Write a one-sentence description in your own words for each of the functions. We will use those functions in step 8.

## **Step 8) Use all functions and evaluate every drug in every feature combination!**

Alright this is the part where we put all our created functions into use by creating a for loop, that chains each of the steps we have done so far.



**Complete Task 2 here**

#### **a) Lets recall the list of drugs we have available and the combination of features we are interested in**

In [ ]:
# let's check all drugs
drug_list

In [ ]:
# let's see all combinations we are interested in
combo_list

#### **b) Create a loop that will go through all our functions using the lists above**

Below is how we chose to chain these functions in order to get all our results and store them in a dictionary called **GB_model_metrics**. Note that this will take a long time as it is training for each drug every combination of features we have specified, so just sit back and grab something to drink as the computer does it's job.  You can check the print out to see what model it's currently training.

**Note:** This loop will take some time to run. Lucy Moctezuma who wrote the code says it took 42 minutes when she last did it.
If that is too long for you, feel free to focus on fewer of the drugs and fewer of the combintions. For example, you could loop over part of the drug_list (for drug in drug_list[:3]

In [ ]:
# Lets use all our functions this time and save our report into a single data structure
GB_model_metrics = {}

for drug in drug_list:
  print(drug)
  Drug_df = makeDF(drug) # creates one df per drug
  Test_Train_dic = Split_train_test(Drug_df, drug) # splits each drug df into a dictionary with testing and training data
  for combo in combo_list:
    # Training each drug_combo features
    labels_train = Test_Train_dic["labels_train"]
    features_train = combo_feat(Test_Train_dic["features_train"], drug, combo) # create corresponding feature_df for training
    GB_combo_model = run_GB(features_train, labels_train, drug, combo) # runs gradient boosted model using the corresponding training feature_df

    # Predicting each drug_combo features
    features_test = combo_feat(Test_Train_dic["features_test"], drug, combo) # create corresponding feature_df for testing
    labels_pred = predict(GB_combo_model, features_test) # generate predictions based on the feature combination tested

    # Evaluating our models
    labels_test = Test_Train_dic["labels_test"]
    report = evaluate(GB_combo_model, labels_test, labels_pred, cf=False) #extracting the metrics we want from the models report
    GB_model_metrics[drug+"_"+combo] = report #saving these metrics into dictionary

    print(report)

#### **c) Store the metrics report for all drugs and features combinations as a csv file**

After running our code is only necessary for us to save it as a csv file, that way we can access it later on!

In [ ]:
# convert dictionary into a dataframe
GB_metrics = pd.DataFrame.from_dict(GB_model_metrics, orient='index',columns=["Accuracy", "R_recall", "S_recall"]).reset_index()
GB_metrics = GB_metrics.rename(columns = {'index':'Drug_combo'})

# saving our metric results into a CSV file
GB_metrics.to_csv(filepath+"GB_metrics_df.csv", index= False)
GB_metrics


#### **d) Create a bar graph showing accuracies of all drugs when using all features (GS)**

Below we will create a quick bar graph checking at how each of the drugs performed using the specific combination of GS. (Gene Absence and Presence + Population Structure)

In [ ]:
# filtering for all the rows that contain GS combination only
GS_filter = [drug_combo for drug_combo in GB_metrics['Drug_combo'] if drug_combo.endswith("GS")]
GS_df = GB_metrics.loc[GB_metrics["Drug_combo"].isin(GS_filter)]


In [ ]:
# plotting bar graph of only

# Figure Size
fig = plt.figure(figsize =(20, 8))

# Adding title
plt.title('Accuracy, R_recall and S_recall', fontsize = 12)

# Variables to be plotted
x = np.arange(len(GS_df["Drug_combo"]))
acc = list(GS_df["Accuracy"])
R_rec = list(GS_df["R_recall"])
S_rec = list(GS_df["S_recall"])

# Plotting barcharts
acc_bar=plt.bar(x-0.25, height= acc, width=0.25, color="darkgrey", edgecolor="gray")
r_rec_bar=plt.bar(x, height= R_rec, width=0.25, color="goldenrod", align="center", edgecolor="gray")
s_rec_bar=plt.bar(x+0.25, height= S_rec, width=0.25, color="palegoldenrod", edgecolor="gray")

plt.xticks([r for r in range(len(GS_df["Drug_combo"]))],
        GS_df["Drug_combo"], fontsize = 12)

#legend
fig.legend([acc_bar,r_rec_bar,s_rec_bar],["Accuracy", "R_recall", "S_recall"], bbox_to_anchor=(0.4,-0.35, 0.04, 0.4), fontsize=12)

# Show Plot
plt.show()


## Task 3:

Looking at the Graph above and the barplots for percentage resistance you made in 6A, why do you think TZP had such a high accuracy but such low Resistance Recall Score?

**Answer your question here**

## Task 4. Optional here but will be required in Module 7:

Now that you have seen how creating different functions and stringing them together allows you to tackle a more complex project, try to challenge yourself and do the same but using a Random Forest Model!
- Feel free to use this code as a general guide, but remember there are many ways to tackle this problem!
- Create your own functions and see if you can get as an end result to recreate the final dataframe and csv file with all the results stored.
  - If you choose to do this challenge now, name this file "**RF_metrics_df.csv**" (like your GB_metrics_df.csv file created here)
    - You will need to create the RF_metrics_df.csv file for Notebook 7b if you do not do it here.